# Blood Bowl AI Training
## Setup & Train on Google Colab

1. **Setup** (~3 min): Install deps, clone repo, build C++ engine
2. **Train**: Run training with macro-MCTS policy
3. **Download**: Get trained weights

In [ ]:
# Cell 1: Install dependencies & build engine
!apt-get update -qq && apt-get install -y -qq cmake g++ > /dev/null 2>&1
!pip install -q pybind11 numpy

# Clone repo
!git clone -q https://github.com/jansekera/bloodbowl.git 2>/dev/null || echo 'Already cloned'

# Build C++ engine
import os
os.chdir('/content/bloodbowl')

!mkdir -p engine/build
pybind_dir = !python3 -c "import pybind11; print(pybind11.get_cmake_dir())"
!cd engine/build && cmake .. -DCMAKE_BUILD_TYPE=Release -Dpybind11_DIR={pybind_dir[0]} -DPYTHON_EXECUTABLE=$(which python3) -DCMAKE_CXX_FLAGS="-O3" > /dev/null 2>&1 && make -j$(nproc) 2>&1 | tail -3

# Verify
import sys
sys.path.insert(0, 'engine/build')
sys.path.insert(0, 'python')
import bb_engine
print(f'Engine loaded: {len(bb_engine.get_all_roster_names())} rosters')

In [ ]:
# Cell 2: Configure training
# Change these parameters as needed:

EPOCHS = 5
GAMES_PER_EPOCH = 20
OPPONENT = 'greedy'       # 'random' or 'greedy'
HOME_RACE = 'human'       # or 'orc', 'skaven', etc.
AWAY_RACE = 'orc'         # or 'skaven', 'dwarf', 'wood-elf'
MCTS_ITERATIONS = 400
EPSILON_START = 0.15
EPSILON_END = 0.05
BENCHMARK_INTERVAL = 5    # benchmark every N epochs

print(f'Training: {EPOCHS} epochs x {GAMES_PER_EPOCH} games')
print(f'Matchup: {HOME_RACE} (macro_mcts) vs {AWAY_RACE} ({OPPONENT})')
print(f'MCTS: {MCTS_ITERATIONS} iterations')

In [ ]:
# Cell 3: Run training
os.chdir('/content/bloodbowl')
os.environ['PYTHONPATH'] = 'engine/build:python'

!python3 -m blood_bowl.train_cli \
    --epochs={EPOCHS} --games={GAMES_PER_EPOCH} --use-cpp \
    --opponent={OPPONENT} \
    --home-race={HOME_RACE} --away-race={AWAY_RACE} \
    --mcts-iterations={MCTS_ITERATIONS} --policy-lr=0.01 \
    --lr=0.0 --model=linear \
    --weights=weights_best.json \
    --training-method=mc_shaped --gamma=0.99 \
    --epsilon-start={EPSILON_START} --epsilon-end={EPSILON_END} \
    --benchmark-interval={BENCHMARK_INTERVAL} --benchmark-matches=15 \
    --skip-greedy-benchmark --timeout=300

In [ ]:
# Cell 4: Push results to GitHub (needs token)
# Option A: Download weights manually
from google.colab import files
files.download('weights_best.json')
files.download('weights_train.json')

In [ ]:
# Cell 5 (optional): Push to GitHub with personal access token
# Create token at: github.com/settings/tokens (repo scope)
# Uncomment and fill in:

# GITHUB_TOKEN = 'ghp_...'  # paste your token here
# !git remote set-url origin https://{GITHUB_TOKEN}@github.com/jansekera/bloodbowl.git
# !git add weights_best.json
# !git commit -m 'Update weights from Colab training'
# !git push